In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font

C:\Users\Christina\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Christina\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\Christina\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
base_link = "https://www.rescoop.eu/network"
data = []

for i in np.arange(1, 60):
    url = base_link + "/p%d" % (i)
    response = requests.get(url)

    if response.status_code != 200:  # Checking if the page is accessible
        continue

    soup = BeautifulSoup(response.content, 'html.parser')

    articles = soup.select('.article-card.network-item')

    for article in articles:
        org_name = article.select_one('.article-content h2').text
        location = article.select_one('footer h4').text
        active_in_elements = article.select('.term-list li a')
        active_in_data = ", ".join([elem.text for elem in active_in_elements])
        website_elem = article.select_one('.buttons .button.external')
        website_url = website_elem['href'] if website_elem else 'N/A'

        data.append({
            "Organization Name": org_name,
            "Location": location,
            "Active In": active_in_data,
            "Website": website_url
        })

df = pd.DataFrame(data)
#df
#df.to_csv("data/network_rescoop.csv", index=False)

wb = Workbook()
ws = wb.active

for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        if cell.column_letter == 'D' and (str(value).startswith("http://") or str(value).startswith("https://")):
            cell.hyperlink = value  # Set the hyperlink attribute
            cell.font = Font(color="0000FF", underline="single")

wb.save("data/network_rescoop.xlsx")